In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# change path if necessary
import sys
my_path = r'D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project2'
sys.path.insert(0,my_path + r'/code/COMMON')

# imports
import numpy as np 
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Glove model

In [5]:
from dictionary_helpers import build_glove_dict

In [6]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/twitter_datasets_stanford/')

In [7]:
filename_glove_dict = 'glove.twitter.27B.25d.txt'

glove = build_glove_dict(filename_glove_dict)

# Load stop words

In [8]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/stop_words/')

In [9]:
filename_stopwords = 'stop_word_freq_min_100_ratio_marg_0.1.txt'

stop_words = []
with open(filename_stopwords, 'r', encoding='utf-8-sig') as f:
    for line in f:
        stop_words.append(line.lstrip().split()[0])
    del stop_words[-1]
    
print("File :", filename_stopwords)
print("Number of stop words :", len(stop_words))

File : stop_word_freq_min_100_ratio_marg_0.1.txt
Number of stop words : 630


# Load tfid 

In [10]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/tfidf/')

In [11]:
filename_tfidf = 'tfidf.txt'

tfidf = {}
with open(filename_tfidf, 'r', encoding='utf-8-sig') as f:
    next(f) # skip headers
    for line in f:
        word = line.strip().split()[0]
        tf = float(line.strip().split()[1])
        idf = float(line.strip().split()[2])
        tfidf[word] = [tf, idf]

# Build tweet vector method

In [12]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/code/tom/')

In [13]:
from tweet_processings import build_tweet_vector

In [14]:
# method to build tweet vector
method = ["mean"]

# Build tweet vectors TRAIN

In [15]:
import os
os.chdir(r'D:\Documents\etudes\epfl\MA1\cours\MachineLearning\Project2\data\twitter_datasets_epfl\short')

In [16]:
# build positive tweet feature set
X_pos = []

with open('train_pos_processed.txt') as f:
    for line in f:
        tweet = line.lstrip().split()
        tweet_vector = build_tweet_vector(tweet, glove, tfidf, stop_words, method)
        if len(tweet_vector):
            X_pos.append(tweet_vector)
        
X_pos = np.array(X_pos)

'lisining to music' is an empty tweet.
'off to burgerboys' is an empty tweet.
'walmart with mommiiee' is an empty tweet.
'off to walmart' is an empty tweet.
'up with andy' is an empty tweet.
'with deanza' is an empty tweet.
'going to crowely' is an empty tweet.


In [17]:
# build negative tweet feature set
X_neg = []

with open('train_neg_processed.txt') as f:
    for line in f:
        tweet = line.lstrip().split()
        tweet_vector = build_tweet_vector(tweet, glove, tfidf, stop_words, method)
        if len(tweet_vector):
            X_neg.append(tweet_vector)
        
X_neg = np.array(X_neg) 

'enough' is an empty tweet.
'im thinking too much' is an empty tweet.
'kgnbgtini hugmepleaseprince ~' is an empty tweet.
'scoiled my tongue' is an empty tweet.
'so insanly busy' is an empty tweet.
'<number><number> philly' is an empty tweet.
'willy willy willy willy' is an empty tweet.


In [18]:
# labels
y_pos = np.ones(X_pos.shape[0])
y_neg = -np.ones(X_neg.shape[0])

In [19]:
# number of training samples
N_samples_train = 50000

# cut samples
X_pos_cut = X_pos[:N_samples_train,:]
X_neg_cut = X_neg[:N_samples_train,:]

# cut targets
y_pos_cut = y_pos[:N_samples_train]
y_neg_cut = y_neg[:N_samples_train]

# concatenate
X_pos_neg = np.concatenate([X_pos_cut, X_neg_cut])
y_pos_neg = np.concatenate([y_pos_cut, y_neg_cut])

# Standardization

In [22]:
# set to "True" to standardize
ifStandardize = False

In [23]:
from sklearn.preprocessing import StandardScaler

if ifStandardize:
    
    scaler = StandardScaler()
    scaler.fit(X_pos_neg)
    X_pos_neg = scaler.transform(X_pos_neg)
    
else: 
    
    X_pos_neg = X_pos_neg

# Hyperparameter optimization (K)

## Grid search and cross validation

In [25]:
from __future__ import print_function

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

# data
X = X_pos_neg
y = y_pos_neg

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# range for hyperparameters
K_range = range(1,10)

# Set the parameters by cross-validation
tuned_parameters = [{'n_neighbors': K_range}]
                    
# define grid search CV
clf = GridSearchCV(KNeighborsClassifier(), tuned_parameters, cv=5, scoring= 'accuracy')

# fit for every parameters combinations in grid search CV
clf.fit(X_train, y_train)

KeyboardInterrupt: 

## Display results

In [ ]:
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()

means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
    
print()
print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

In [2]:
from matplotlib.colors import Normalize

# Utility function to move the midpoint of a colormap to be around
# the values of interest.

class MidpointNormalize(Normalize):

    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))
   
scores = clf.cv_results_['mean_test_score'].reshape(len(C_range), len(gamma_range))

plt.figure(figsize=(8, 6))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores, interpolation='nearest', cmap=plt.cm.hot, norm=MidpointNormalize(vmin=0.2, midpoint=0.92))
plt.xlabel('gamma')
plt.ylabel('C')
plt.colorbar()
plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
plt.yticks(np.arange(len(C_range)), C_range)
plt.title('Validation accuracy')
plt.show()

NameError: name 'clf' is not defined